In [1]:
import numpy as np
import gym
import tensorflow as tf
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, Input

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) # stop displaying warnings that mean nothing!!!

/Users/captain/opt/anaconda3/lib/python3.8/site-packages/ale_py/roms/__init__.py:94: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  _RESOLVED_ROMS = _resolve_roms()
/Users/captain/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:585: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/Users/captain/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/fr

In [2]:
def prepro(I):
    # preprocess each frame for learning
    # save some memory and computation
    # pre-process the image from a 210x160x3 uint8 frame into an (80x80) float array 
    I = I[35:195,:,:].copy() # crop the top of the image...score image doesn't matter for how to play
    I = I[::2,::2,0].copy()
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return np.array(I.copy())

In [3]:
# keras functional api
# very slow and won't work well
def create_model(height,width,channels):
    # we cannot simply have 3 output nodes because we want to put a weight on each node's impact to the objective
    # that is different for each data point.  the only way to achieve this is to have 3 output layers, each having 1 node
    # the effect is the same, just the way TF/keras handles weights is different
    imp = Input(shape=(height,width,channels))
    mid = Conv2D(16,(8,8),strides=4,activation='relu')(imp)
    mid = Conv2D(32,(4,4),strides=2,activation='relu')(mid)
    mid = Flatten()(mid)
    mid = Dense(256,activation='relu')(mid)
    
    # output layer with 3 nodes does not work
    # we trick tf => 3 output layers & each has one node
    
    out0 = Dense(1,activation='linear',name='out0')(mid)
    out1 = Dense(1,activation='linear',name='out1')(mid)
    out2 = Dense(1,activation='linear',name='out2')(mid)
    model = Model(imp,[out0,out1,out2])  # three output nodes in list
    # why? to set different objective function for each node
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),loss='mean_squared_error')
    
    return model

In [4]:
frames_to_net = 4              # how many previous frames will we feed the NN
possible_actions = [0,2,3]
mod = create_model(80,80,frames_to_net)
mod.call = tf.function(mod.call,experimental_relax_shapes=True)

mod.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 4)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 19, 19, 16)   4112        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 8, 8, 32)     8224        conv2d[0][0]                     
__________________________________________________________________________________________________
flatten (Flatten)               (None, 2048)         0           conv2d_1[0][0]                   
______________________________________________________________________________________________

2022-04-29 08:44:07.047509: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def play1game(model,ep):
    env0 = gym.make("Pong-v0")
    pix = env0.reset()
    pix = prepro(pix)
    frames_this_game = 0
    feed = np.zeros((1,80,80,frames_to_net))
    feed[0,:,:,0] = pix.copy()
    
    frame_array = []
    action_array = []
    reward_array = []
    
    score = 0
    done = False
    while not done:
        if np.random.random() < ep: # epsilon greedy instead of complete random
            action = np.random.choice(3)
        else:
            vf = mod(feed,training=False) # recent 4 frames
            vf = [vf[0][0,0].numpy(),vf[1][0,0].numpy(),vf[2][0,0].numpy()] #0: most recent frame
            action = np.argmax(vf)
        action0 = possible_actions[action]
        pix_new, reward, done, info = env0.step(action0)
        frame_array.append(pix)
        action_array.append(action)
        reward_array.append(reward)
        pix = prepro(pix_new)
        frames_this_game += 1

        for f in range(1,frames_to_net):
            feed[0,:,:,frames_to_net-f] = feed[0,:,:,frames_to_net-f-1].copy()
        feed[0,:,:,0] = pix.copy()
        score += reward
    return frame_array, action_array, reward_array, score

In [6]:
frames, actions, rewards, score = play1game(mod,0.5)

/Users/captain/opt/anaconda3/lib/python3.8/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v5` with the environment ID `ALE/Pong-v5`.
  logger.warn(
A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]
/Users/captain/opt/anaconda3/lib/python3.8/site-packages/gym/utils/seeding.py:138: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/Users/captain/opt/anaconda3/lib/python3.8/site-packages/gym/utils/seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
2022-04-29 08:44:07.612505: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
/Users/captain/opt/anaconda3/lib/python3.8/site-packages/gym/utils/seeding.p

In [7]:
score # we lost all points

-21.0

In [8]:
len(frames) # how many frames in 1 game

1101

In [9]:
ngames = 1000 # let tf play 1000 games
epsvec = np.linspace(1,0.05,ngames) # decrease epsilon
ngames = 100 # lets just play 100 
delt = 0.99 # discount rate
nbatch = 32 # for sgd

In [17]:
records = []
for game in range(ngames):
    start = time.time()
    frames, actions, rewards, score = play1game(mod,epsvec[game])

    nframes = len(frames) # how many frames were there
    current_frames = np.zeros((nframes,80,80,frames_to_net)) # create tensor, I am going to plug recent 4 frames
    future_frames = np.zeros((nframes,80,80,frames_to_net))  # for vector of truth (pretend it as truth)
    # if time 7, then 5 6 7 8
  
    for grab in range(nframes):
        for f in range(frames_to_net):
            if grab-f > 0:
                current_frames[grab,:,:,f] = frames[grab-f].copy()
            if (grab-f+1 > 0) & (grab-f+1 < (nframes-1)):
                future_frames[grab,:,:,f] = frames[grab-f+1].copy()
    target_vf = mod.predict(future_frames) # predicted truth (y hat)
    
    # three vectors of truth
    y0 = np.zeros((nframes,1))
    y1 = np.zeros((nframes,1))
    y2 = np.zeros((nframes,1))
    
    # three vectors of weight (1 for pushed, 0 else)
    weight0 = np.zeros(nframes)
    weight1 = np.zeros(nframes)
    weight2 = np.zeros(nframes)
  
    for grab in range(nframes):
        rhs = rewards[grab] # lets say truth is reward for now
        if rhs == 0: # if reward was 0, discounted value
            rhs = delt*np.max([target_vf[0][grab],target_vf[1][grab],target_vf[2][grab]])
        # assign weights
        if actions[grab] == 0:
            y0[grab,0] = rhs
            weight0[grab] = 1
        elif actions[grab] == 1:
            y1[grab,0] = rhs
            weight1[grab] = 1
        else:
            y2[grab,0] = rhs
            weight2[grab] = 1
  
    mod.fit(current_frames,[y0,y1,y2],epochs=1,batch_size=nbatch,verbose=0,sample_weight={'out0':weight0,'out1':weight1,'out2':weight2},use_multiprocessing=True)
    stop = time.time()
    print([game, score, epsvec[game], stop-start])
    records.append(score)


[0, -21.0, 1.0, 3.080907106399536]
[1, -21.0, 0.999049049049049, 2.314507007598877]
[2, -21.0, 0.9980980980980981, 1.7715950012207031]
[3, -21.0, 0.9971471471471471, 1.779400110244751]
[4, -20.0, 0.9961961961961961, 2.2440710067749023]
[5, -20.0, 0.9952452452452453, 2.351041078567505]
[6, -19.0, 0.9942942942942943, 2.3224940299987793]
[7, -21.0, 0.9933433433433434, 2.023738145828247]
[8, -21.0, 0.9923923923923924, 1.86279296875]
[9, -20.0, 0.9914414414414414, 2.35799503326416]
[10, -21.0, 0.9904904904904905, 2.4658091068267822]
[11, -20.0, 0.9895395395395395, 2.3893508911132812]
[12, -21.0, 0.9885885885885886, 1.7554800510406494]
[13, -21.0, 0.9876376376376377, 1.8032748699188232]
[14, -21.0, 0.9866866866866867, 1.7104051113128662]
[15, -21.0, 0.9857357357357357, 1.6689338684082031]
[16, -21.0, 0.9847847847847848, 2.104820966720581]
[17, -21.0, 0.9838338338338338, 1.6429939270019531]
[18, -21.0, 0.9828828828828828, 1.5947740077972412]
[19, -21.0, 0.981931931931932, 1.6222240924835205]


In [18]:
# This simple method does not work well
print(f'The average score: {sum(records)/len(records)}')

The average score: -20.65
